# AIDS Incident Cases In Illinois Department of Publich Health
A dataset that is extracted real time from a website, saved as csv format, Uploaded to the sql database for storage.
This data is based on AIDS cases on the Illinois department of Public Health. It has 3 tables.
1) Aids cases on every county
2) Highest county Aids Cases breakdown
3) Second Highest County Aids Cases Breakdown

In [1]:
import time
import csv
import mysql.connector
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common import NoSuchElementException
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from dash import Dash, dcc, html, Input, Output
%matplotlib inline
from jupyter_datatables import init_datatables_mode
from plotly.offline import download_plotlyjs, plot, init_notebook_mode,iplot
init_notebook_mode(connected=True)

In [4]:
# Using selenium to extract real time data for the Aids cases on every county
driver = webdriver.Chrome()
driver.maximize_window
driver.get(
    url='https://dph.illinois.gov/topics-services/diseases-and-conditions/hiv-aids/hiv-surveillance/update-reports/2023/february.html')

# waiting for table to load
TableWait = WebDriverWait(driver, 10)
Table = TableWait.until(EC.presence_of_element_located((By.XPATH, "//table[@id='DataTables_Table_10']")))

# get the number of pages on the table
table_text = driver.find_element(By.CSS_SELECTOR, '#DataTables_Table_10_info').text
start_index = table_text.find('of') + 3
last_index = table_text.find('entries') - 1
No_entries =int(table_text[start_index:last_index])
print(f"No Of entries: {No_entries}")

data=[]
for No_entries in range(1, No_entries +1):
    try:
        Table = TableWait.until(EC.presence_of_element_located((By.XPATH, "//table[@id='DataTables_Table_10']")))
        
        # finding elements on the table
        DataRow = Table.find_elements(By.XPATH, ".//tbody/tr")
        for _ in DataRow:
            cells = _.find_elements(By.XPATH, ".//td")
            row = [cell.text for cell in cells]
            data.append(row)

        # checking if the last entry on the table is Illinois
        if any('Illinois' in DataRow for DataRow in data):
            break

        # next element button
        next_element = driver.find_element(By.XPATH, "//a[@id='DataTables_Table_10_next']")
        if 'disabled' in next_element.get_attribute('class'):
            break
        next_element.click()

        time.sleep(10)

    except NoSuchElementException as e:
        print(f'Error: {e}')
        break
        
for row in data:
    print(row)
        
time.sleep(20)
driver.quit()

# saving the data to csv
with open('Aids_Incident_Cases_In_Illinois.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    if data:
        writer.writerow(['County', 'Cases Diagnosed As Of 2/28/2023', 
                         'Cumulative Cases Diagnosed Since 2016', '2016-2023 AIDS Diagnosis Rate'])
    writer.writerows(data)

print()
print('Data successfully Saved as Csv!')

No Of entries: 104
['Adams', '0', '6', '1.25']
['Alexander', '0', '2', '3.6']
['Bond', '0', '1', '0.79']
['Boone', '1', '3', '0.78']
['Brown', '0', '1', '2.02']
['Bureau', '0', '1', '0.41']
['Calhoun', '0', '0', '0']
['Carroll', '0', '0', '0']
['Cass', '0', '2', '2.09']
['Champaign', '0', '45', '3.09']
['Christian', '0', '4', '1.61']
['Clark', '0', '0', '0']
['Clay', '0', '2', '2.03']
['Clinton', '0', '3', '1.1']
['Coles', '0', '3', '0.78']
['Cook', '46', '2898', '7.73']
['Crawford', '0', '4', '2.85']
['Cumberland', '0', '1', '1.27']
['DeKalb', '2', '17', '2.27']
['De Witt', '0', '2', '1.7']
['Douglas', '0', '0', '0']
['DuPage', '2', '100', '1.5']
['Edgar', '0', '0', '0']
['Edwards', '0', '0', '0']
['Effingham', '2', '5', '2.03']
['Fayette', '1', '7', '4.44']
['Ford', '0', '0', '0']
['Franklin', '0', '9', '3.19']
['Fulton', '0', '6', '2.28']
['Gallatin', '0', '1', '2.57']
['Greene', '1', '1', '1.03']
['Grundy', '0', '3', '0.83']
['Hamilton', '0', '1', '1.67']
['Hancock', '0', '1', '0.7

In [9]:
# Using selenium to extract real time data for the Highest county Aids cases breakdown
browser = webdriver.Chrome()
browser.maximize_window
browser.get(
    url='https://dph.illinois.gov/topics-services/diseases-and-conditions/hiv-aids/hiv-surveillance/update-reports/2023/february.html')

# getting the table details
tabledetails = browser.find_element(By.ID, "DataTables_Table_2_wrapper")
rowdetails = tabledetails.find_elements(By.XPATH, ".//tbody/tr")
row_count = len(rowdetails)
print(f'No of Rows: {row_count}')

data = []
for _ in rowdetails:
    cells =_.find_elements(By.XPATH, ".//td")
    row_cell =[cell.text for cell in cells]
    data.append(row_cell)

for row in data:
    print(row)

# Saving the data on a csv file
with open('Highest county Aids cases breakdown.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    if data:
        writer.writerow(['Health Department', 'Cases Diagnosed As Of 2/28/2023', 'Cumulative Cases Diagnosed Since 2016'])
    writer.writerows(data)
print('Data saved to csv format successfully')    

No of Rows: 5
['Chicago Health Dept.', '40', '2177']
['Cook Co. Health Dept.', '6', '677']
['Evanston Health Dept.', '0', '21']
['Oak Park Health Dept.', '0', '13']
['Skokie Health Dept.', '0', '10']
Data saved to csv format successfully


In [10]:
df = pd.read_csv('Aids_Incident_Cases_In_Illinois.csv')

In [11]:
df

,County,Cases Diagnosed As Of 2/28/2023,Cumulative Cases Diagnosed Since 2016,2016-2023 AIDS Diagnosis Rate
0,Adams,0,6,1.25
1,Alexander,0,2,3.60
2,Bond,0,1,0.79
3,Boone,1,3,0.78
4,Brown,0,1,2.02
...,...,...,...,...
99,Williamson,0,4,0.84
100,Winnebago,1,61,2.91
101,Woodford,0,3,1.07
102,Unknown,0,1,NaN
